In [ ]:
#https://medium.com/@lope.ai/recommendation-systems-from-scratch-in-python-pytholabs-6946491e76c2
#https://jonathan-hui.medium.com/machine-learning-recommender-system-e3237b9df14a

In [ ]:
D'abord, récolter les données, faire du preprocessing sur les données,(scraping ou dataset déjà fait)
 choisir le meilleure modèle (dans le cas du NLP, CountVectorizer ou TF-IDFVectorizer associé 
à la cosine similarity), coder une fonction permettant
 d'obtenir les meilleurs recommandations pour un élément donné en entrée. En gros, c'est ça. 

In [ ]:
#Le but du projet est de faire un systemes de recomendation de recette de cuisine pour utiliser le produit  qui va devenir périsable  
#  l'utilisateur  va prendre  une photos de  tous c'est  produit dans sont frigidaire.
# puis la machine va mémoriser tous les dates de pérunption  
# chaque  fois que  la  date approche de deux  jours va poposer une  recette de  gateau de  plat ou d'entré pour  les produit qui vont se périmer     

In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
print("fait")

fait


Analyse  des données  du dataset  de  film pour  l'étude  de  systemes recommendation  

In [ ]:
df_movie=pd.read_csv('/content/sample_data/mdb_5000_credits.csv', sep=',')
df_movie=pd.read_csv('/content/sample_data/mdb_5000_movies.csv', sep=',')

In [ ]:
df_movie.dtypes

budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
dtype: object

In [ ]:
print('Les colonnes de tmdb_5000_movies sont',df_movie.columns)

Les colonnes de tmdb_5000_movies sont Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [ ]:
reco=df_movie.drop(columns=['tagline','original_title',"id","original_language", "homepage",'original_language','status','keywords','release_date','spoken_languages','budget','revenue','production_companies','production_countries','genres','popularity','runtime','vote_average','vote_count'])
reco

,overview,title
0,"In the 22nd century, a paraplegic Marine is di...",Avatar
1,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,A cryptic message from Bond’s past sends him o...,Spectre
3,Following the death of District Attorney Harve...,The Dark Knight Rises
4,"John Carter is a war-weary, former military ca...",John Carter
...,...,...
4798,El Mariachi just wants to play his guitar and ...,El Mariachi
4799,A newlywed couple's honeymoon is upended by th...,Newlyweds
4800,"""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered"
4801,When ambitious New York attorney Sam is sent t...,Shanghai Calling


In [ ]:
print('Les colonnes de tmdb_5000_movies sont',reco.columns)

Les colonnes de tmdb_5000_movies sont Index(['overview', 'title'], dtype='object')


In [ ]:
reco.describe().transpose()

,count,unique,top,freq
overview,4800,4800,Rock Star tells the story of Chris Cole and a ...,1
title,4803,4800,Batman,2


# **preprocessing**

In [ ]:
reco

,overview,title
0,"In the 22nd century, a paraplegic Marine is di...",Avatar
1,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,A cryptic message from Bond’s past sends him o...,Spectre
3,Following the death of District Attorney Harve...,The Dark Knight Rises
4,"John Carter is a war-weary, former military ca...",John Carter
...,...,...
4798,El Mariachi just wants to play his guitar and ...,El Mariachi
4799,A newlywed couple's honeymoon is upended by th...,Newlyweds
4800,"""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered"
4801,When ambitious New York attorney Sam is sent t...,Shanghai Calling


In [ ]:
pip  install  nltk

In [ ]:
!pip install stopwords

     |████████████████████████████████| 51kB 2.2MB/s 
  Created wheel for stopwords: filename=stopwords-0.1.3-py2.py3-none-any.whl size=37286 sha256=42e5c6f4bffafa896672eabac8e453485cd025a93df34ea415cbce7874120f0b
  Stored in directory: /root/.cache/pip/wheels/39/fa/c7/c4c5111e658f5c58465d948165dc3395a3c10ff57f4cd20356
Successfully built stopwords


In [ ]:
!pip install unicode

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
reco['title'] = reco['title'].str.lower()
reco['overview'] = reco['overview'].str.lower()

In [ ]:
reco.isnull().sum()

overview    3
title       0
dtype: int64

In [ ]:
reco

,overview,title
0,"in the 22nd century, a paraplegic marine is di...",avatar
1,"captain barbossa, long believed to be dead, ha...",pirates of the caribbean: at world's end
2,a cryptic message from bond’s past sends him o...,spectre
3,following the death of district attorney harve...,the dark knight rises
4,"john carter is a war-weary, former military ca...",john carter
...,...,...
4798,el mariachi just wants to play his guitar and ...,el mariachi
4799,a newlywed couple's honeymoon is upended by th...,newlyweds
4800,"""signed, sealed, delivered"" introduces a dedic...","signed, sealed, delivered"
4801,when ambitious new york attorney sam is sent t...,shanghai calling


In [ ]:
df_drop1=df_movie.drop(columns=["tagline","original_title",	"id","original_language", "homepage",'status','keywords','release_date','spoken_languages','budget','revenue','production_companies','production_countries','genres','overview','tagline','title'	])
df_drop1

,popularity,runtime,vote_average,vote_count
0,150.437577,162.0,7.2,11800
1,139.082615,169.0,6.9,4500
2,107.376788,148.0,6.3,4466
3,112.312950,165.0,7.6,9106
4,43.926995,132.0,6.1,2124
...,...,...,...,...
4798,14.269792,81.0,6.6,238
4799,0.642552,85.0,5.9,5
4800,1.444476,120.0,7.0,6
4801,0.857008,98.0,5.7,7


In [ ]:
df_drop1.isnull().sum()

popularity      0
runtime         2
vote_average    0
vote_count      0
dtype: int64

In [ ]:
concat_df = pd.concat([df_drop1, reco], axis=1)
concat_df

,popularity,runtime,vote_average,vote_count,overview,title
0,150.437577,162.0,7.2,11800,"in the 22nd century, a paraplegic marine is di...",avatar
1,139.082615,169.0,6.9,4500,"captain barbossa, long believed to be dead, ha...",pirates of the caribbean: at world's end
2,107.376788,148.0,6.3,4466,a cryptic message from bond’s past sends him o...,spectre
3,112.312950,165.0,7.6,9106,following the death of district attorney harve...,the dark knight rises
4,43.926995,132.0,6.1,2124,"john carter is a war-weary, former military ca...",john carter
...,...,...,...,...,...,...
4798,14.269792,81.0,6.6,238,el mariachi just wants to play his guitar and ...,el mariachi
4799,0.642552,85.0,5.9,5,a newlywed couple's honeymoon is upended by th...,newlyweds
4800,1.444476,120.0,7.0,6,"""signed, sealed, delivered"" introduces a dedic...","signed, sealed, delivered"
4801,0.857008,98.0,5.7,7,when ambitious new york attorney sam is sent t...,shanghai calling


#  algorithme TIDF

In [ ]:
C = concat_df['vote_average'].mean()
print(C)

6.092171559442011


In [ ]:
m = concat_df['vote_count'].quantile(0.90)
print(m)

1838.4000000000015


In [ ]:
q_movies = concat_df.copy().loc[concat_df['vote_count'] >= m]
q_movies

,popularity,runtime,vote_average,vote_count,overview,title
0,150.437577,162.0,7.2,11800,"in the 22nd century, a paraplegic marine is di...",avatar
1,139.082615,169.0,6.9,4500,"captain barbossa, long believed to be dead, ha...",pirates of the caribbean: at world's end
2,107.376788,148.0,6.3,4466,a cryptic message from bond’s past sends him o...,spectre
3,112.312950,165.0,7.6,9106,following the death of district attorney harve...,the dark knight rises
4,43.926995,132.0,6.1,2124,"john carter is a war-weary, former military ca...",john carter
...,...,...,...,...,...,...
4291,63.655973,103.0,7.2,2184,obsessed with teaching his victims the value o...,saw
4300,66.925866,99.0,8.0,3697,a botched robbery indicates a police informant...,reservoir dogs
4302,88.377076,161.0,8.1,2311,while the civil war rages between the union an...,"the good, the bad and the ugly"
4337,58.845025,114.0,8.0,2535,a mentally unstable vietnam war veteran works ...,taxi driver


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies

,popularity,runtime,vote_average,vote_count,overview,title,score
0,150.437577,162.0,7.2,11800,"in the 22nd century, a paraplegic marine is di...",avatar,7.050669
1,139.082615,169.0,6.9,4500,"captain barbossa, long believed to be dead, ha...",pirates of the caribbean: at world's end,6.665696
2,107.376788,148.0,6.3,4466,a cryptic message from bond’s past sends him o...,spectre,6.239396
3,112.312950,165.0,7.6,9106,following the death of district attorney harve...,the dark knight rises,7.346721
4,43.926995,132.0,6.1,2124,"john carter is a war-weary, former military ca...",john carter,6.096368
...,...,...,...,...,...,...,...
4291,63.655973,103.0,7.2,2184,obsessed with teaching his victims the value o...,saw,6.693677
4300,66.925866,99.0,8.0,3697,a botched robbery indicates a police informant...,reservoir dogs,7.366378
4302,88.377076,161.0,8.1,2311,while the civil war rages between the union an...,"the good, the bad and the ugly",7.210428
4337,58.845025,114.0,8.0,2535,a mentally unstable vietnam war veteran works ...,taxi driver,7.198026


In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)
#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
1881,the shawshank redemption,8205,8.5,8.059258
662,fight club,9413,8.3,7.939256
65,the dark knight,12002,8.2,7.920020
3232,pulp fiction,8428,8.3,7.904645
96,inception,13752,8.1,7.863239
3337,the godfather,5893,8.4,7.851236
95,interstellar,10867,8.1,7.809479
809,forrest gump,7927,8.2,7.803188
329,the lord of the rings: the return of the king,8064,8.1,7.727243
1990,the empire strikes back,5879,8.2,7.697884


In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
q_movies['overview'] = q_movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(q_movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(481, 5657)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim 

array([[1.        , 0.00683867, 0.        , ..., 0.0085725 , 0.        ,
        0.03001496],
       [0.00683867, 1.        , 0.        , ..., 0.01888748, 0.        ,
        0.08419618],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0085725 , 0.01888748, 0.        , ..., 1.        , 0.01934522,
        0.04045551],
       [0.        , 0.        , 0.        , ..., 0.01934522, 1.        ,
        0.01860156],
       [0.03001496, 0.08419618, 0.        , ..., 0.04045551, 0.01860156,
        1.        ]])

In [ ]:
cosine_sim.shape

(481, 481)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies['score'] 

1881    8.059258
662     7.939256
65      7.920020
3232    7.904645
96      7.863239
          ...   
41      5.521697
337     5.507643
193     5.459420
91      5.406234
242     5.155730
Name: score, Length: 481, dtype: float64

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
1881,the shawshank redemption,8205,8.5,8.059258
662,fight club,9413,8.3,7.939256
65,the dark knight,12002,8.2,7.920020
3232,pulp fiction,8428,8.3,7.904645
96,inception,13752,8.1,7.863239
3337,the godfather,5893,8.4,7.851236
95,interstellar,10867,8.1,7.809479
809,forrest gump,7927,8.2,7.803188
329,the lord of the rings: the return of the king,8064,8.1,7.727243
1990,the empire strikes back,5879,8.2,7.697884


In [ ]:
concat_df['overview'].head(12)

0     in the 22nd century, a paraplegic marine is di...
1     captain barbossa, long believed to be dead, ha...
2     a cryptic message from bond’s past sends him o...
3     following the death of district attorney harve...
4     john carter is a war-weary, former military ca...
5     the seemingly invincible spider-man goes up ag...
6     when the kingdom's most wanted-and most charmi...
7     when tony stark tries to jumpstart a dormant p...
8     as harry begins his sixth year at hogwarts, he...
9     fearing the actions of a god-like super hero l...
10    superman returns to discover his 5-year absenc...
11    quantum of solace continues the adventures of ...
Name: overview, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
concat_df['overview'] = concat_df['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(concat_df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4803, 20978)

In [ ]:
tfidf.get_feature_names()[5000:5010]

['define',
 'defined',
 'defines',
 'defining',
 'definite',
 'definitely',
 'definition',
 'definitions',
 'deflower',
 'deformed']

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [ ]:
cosine_sim.shape

(4803, 4803)

In [ ]:
cosine_sim[1]

array([0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
       0.        ])

In [ ]:
indices = pd.Series(concat_df.index, index=concat_df['title']).drop_duplicates()
indices

title
avatar                                         0
pirates of the caribbean: at world's end       1
spectre                                        2
the dark knight rises                          3
john carter                                    4
                                            ... 
el mariachi                                 4798
newlyweds                                   4799
signed, sealed, delivered                   4800
shanghai calling                            4801
my date with drew                           4802
Length: 4803, dtype: int64

In [ ]:
indices[:10]

title
avatar                                      0
pirates of the caribbean: at world's end    1
spectre                                     2
the dark knight rises                       3
john carter                                 4
spider-man 3                                5
tangled                                     6
avengers: age of ultron                     7
harry potter and the half-blood prince      8
batman v superman: dawn of justice          9
dtype: int64

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return concat_df['title'].iloc[movie_indices]

In [ ]:
get_recommendations('spider-man 3')

159                   spider-man
30                  spider-man 2
1534               arachnophobia
20        the amazing spider-man
38      the amazing spider-man 2
1318                   the thing
4664                     bronson
3610           not easily broken
4456       raising victor vargas
4276                   def-con 4
Name: title, dtype: object